In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from IPython.display import clear_output
import pandas as pd
import random

In [2]:
df_urls = pd.read_csv('urls_propiedades_en_venta_san_martin_argenprop.csv', index_col=False)
urls = df_urls['0'].tolist()

In [6]:
%%time

exceptions = []
features_global = []

for url in urls[4371:]:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    try:
        features_local = {}
        driver.get(url)

        # Extraccion del precio en USD

        try:
            element = driver.find_element(By.CLASS_NAME, 'titlebar__price').text
            element = element.replace('USD', '')
            element = element.replace('.', '')
            element = element.replace(' ', '')
            features_local['precio_USD'] = int(element)
        except:
            pass

        # Categoria de la propiedad (casa, departamento o ph)

        if 'casa' in url:
            features_local['categoria'] = 'casa'
        elif 'ph' in url: 
            features_local['categoria'] = 'ph'
        elif 'departamento' in url:
            features_local['categoria'] = 'departamento'

        # Extraccion de los elementos principales

        try:
            element = driver.find_element(By.CLASS_NAME, 'property-main-features')
            elements = element.find_elements(By.TAG_NAME, 'p')
        except:
            pass

        try:
            features_local[elements[0].text] = elements[1].text
        except:
            pass

        try:
            features_local[elements[2].text] = elements[3].text
        except:
            pass

        try:
            features_local[elements[4].text] = elements[5].text
        except:
            pass

        try:
            features_local[elements[6].text] = elements[7].text
        except:
            pass

        try:
            features_local[elements[8].text] = elements[9].text
        except:
            pass

        # Correcion de elementos que son equivalentes, son inecesarios o hay que formatear.

        try:
            features_local['Baños'] = features_local['Cantidad de baños']
            del features_local['Cantidad de baños']
        except:
            pass

        try:
            features_local['Baños'] = int(float(features_local['Baños']))
        except:
            pass

        try:
            features_local['Dormitorios'] = features_local['Cantidad de dormitorios']
            del features_local['Cantidad de dormitorios']
        except:
            pass

        try:
            features_local['Dormitorios'] = features_local['Dormitorios'].replace('Monoambiente', '0')
            features_local['Dormitorios'] = int(float(features_local['Dormitorios']))
        except:
            pass

        try:
            features_local['Antiguedad'] = features_local['Antigüedad']
            del features_local['Antigüedad']
        except:
            pass

        try:
            features_local['Antiguedad'] = features_local['Antiguedad'].replace('A Estrenar', '0')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace('A estrenar', '0')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace(' años', '')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace(' año', '')
            features_local['Antiguedad'] = features_local['Antiguedad'].replace(',', '.')
            features_local['Antiguedad'] = int(float(features_local['Antiguedad']))
        except:
            pass

        try:
            features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace('m²', '')
            features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(' ', '')
            features_local['Superficie cubierta'] = features_local['Superficie cubierta'].replace(',', '.')
            features_local['Superficie cubierta'] = float(features_local['Superficie cubierta'])
            features_local['Superficie cubierta'] = int(features_local['Superficie cubierta'])
        except:
            pass

        try:
            del features_local['Disposición']
        except:
            pass

        # Extracion de la latitud y la longitud

        try:
            element = driver.find_element(By.CLASS_NAME, 'map-container')
            element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-latitude')
            features_local['latitude'] = float(element.replace(',','.'))
        except:
            pass

        try:
            element = driver.find_element(By.CLASS_NAME, 'map-container')
            element = element.find_element(By.TAG_NAME, 'div').get_attribute('data-longitude')
            features_local['longitude'] = float(element.replace(',','.'))
        except:
            pass

        # Extraccion del vendedor

        try:
            element = driver.find_element(By.CLASS_NAME, 'details')
            element = element.find_element(By.TAG_NAME, 'h6').text
            features_local['seller'] = element
        except:
            pass


        features_global.append(features_local)

        clear_output(wait=True)

        print(f'Propiedad número {urls.index(url) + 1} extraída ({round(((urls.index(url) + 1)/len(urls))*100 , 2)}%)')

        df = pd.DataFrame(features_global)

        df = df.rename(columns={'precio_USD':'price_USD',
                                'categoria':'type',
                                'Superficie cubierta':'covered_surface_m2',
                                'Dormitorios':'bedrooms',
                                'Baños':'bathrooms',
                                'logitud':'longitude',
                                'Antiguedad':'antiquity'})

        df.to_csv('features_propiedades.csv')
        driver.close()
    except:
        exceptions.append(urls.index(url))
        driver.close()

Propiedad número 4372 extraída (53.98%)



[WDM] - ====== WebDriver manager ======


2022-07-24 01:33:29,734 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 102.0.5005


2022-07-24 01:33:29,864 INFO Current google-chrome version is 102.0.5005


[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome


2022-07-24 01:33:29,868 INFO Get LATEST chromedriver version for 102.0.5005 google-chrome


[WDM] - Driver [/home/nvrancovich/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache


2022-07-24 01:33:30,011 INFO Driver [/home/nvrancovich/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
2022-07-24 01:33:36,358 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fefcf2030d0>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/af8eccb2b586cdd28ec1dddac4eac1df/window
2022-07-24 01:33:36,374 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fefcf1d50a0>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/af8eccb2b586cdd28ec1dddac4eac1df/window
2022-07-24 01:33:36,396 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTT

MaxRetryError: HTTPConnectionPool(host='localhost', port=37803): Max retries exceeded with url: /session/af8eccb2b586cdd28ec1dddac4eac1df/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fefcf1ef670>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price_USD           55 non-null     float64
 1   type                57 non-null     object 
 2   covered_surface_m2  39 non-null     float64
 3   bedrooms            45 non-null     float64
 4   bathrooms           42 non-null     float64
 5   antiquity           36 non-null     float64
 6   latitude            50 non-null     float64
 7   longitude           50 non-null     float64
 8   seller              55 non-null     object 
dtypes: float64(7), object(2)
memory usage: 4.1+ KB


In [17]:
df2.columns

Index(['price_USD', 'type', 'covered_surface_m2', 'bedrooms', 'bathrooms',
       'antiquity', 'latitude', 'longitude', 'seller'],
      dtype='object')

In [28]:
df2 = df
df2

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,latitude,longitude,seller,antiquity
0,160000.0,departamento,104.0,3.0,1.0,-34.575672,-58.535770,GARDEN PROPIEDADES,NaN
1,190000.0,casa,NaN,3.0,NaN,-34.588955,-58.545180,FASSIO ESTUDIO INTEGRAL INMOBILIARIO,NaN
2,268000.0,casa,200.0,4.0,2.0,-34.536670,-58.565130,FASANO PROPIEDADES,33.0
3,64000.0,ph,NaN,2.0,2.0,-34.561893,-58.570590,FASSIO ESTUDIO INTEGRAL INMOBILIARIO,NaN
4,85000.0,departamento,50.0,0.0,1.0,-34.538338,-58.574947,PERRUCCI MENARDI,0.0
...,...,...,...,...,...,...,...,...,...
739,60000.0,ph,NaN,2.0,1.0,-34.568157,-58.541866,Alicia Pinasco inmobiliaria,NaN
740,63000.0,departamento,55.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0
741,69000.0,casa,57.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0
742,69000.0,casa,57.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0


In [30]:
df = pd.read_csv('features_propiedades_3628.csv')
df.drop(axis=1, labels=['Unnamed: 0'], inplace=True)
df[df2.columns]

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,latitude,longitude,seller,antiquity
0,NaN,departamento,44.0,1.0,1.0,NaN,NaN,NaN,6.0
1,122000.0,departamento,53.0,1.0,2.0,-34.545467,-58.552177,Altamirano Propiedades,4.0
2,93500.0,departamento,55.0,1.0,1.0,-34.549740,-58.550430,MANCINI PROPIEDADES,0.0
3,69000.0,departamento,42.0,1.0,1.0,-34.543205,-58.552757,FALUGUE PROPIEDADES,1.0
4,150000.0,departamento,85.0,2.0,2.0,-34.548748,-58.556840,Matias Fortini Propiedades,0.0
...,...,...,...,...,...,...,...,...,...
3623,220000.0,ph,NaN,3.0,1.0,-37.846410,-58.255460,Solimano Dal Pos Inmuebles,0.0
3624,220000.0,departamento,115.0,3.0,2.0,-34.594414,-58.522140,GARDEN PROPIEDADES,NaN
3625,170000.0,casa,130.0,3.0,2.0,-34.557000,-58.542000,GARDEN PROPIEDADES,19.0
3626,132000.0,departamento,110.0,3.0,3.0,-34.575768,-58.540270,GARDEN PROPIEDADES,NaN


In [31]:
df[df2.columns].append(df2).reset_index(drop = True)

,price_USD,type,covered_surface_m2,bedrooms,bathrooms,latitude,longitude,seller,antiquity
0,NaN,departamento,44.0,1.0,1.0,NaN,NaN,NaN,6.0
1,122000.0,departamento,53.0,1.0,2.0,-34.545467,-58.552177,Altamirano Propiedades,4.0
2,93500.0,departamento,55.0,1.0,1.0,-34.549740,-58.550430,MANCINI PROPIEDADES,0.0
3,69000.0,departamento,42.0,1.0,1.0,-34.543205,-58.552757,FALUGUE PROPIEDADES,1.0
4,150000.0,departamento,85.0,2.0,2.0,-34.548748,-58.556840,Matias Fortini Propiedades,0.0
...,...,...,...,...,...,...,...,...,...
4367,60000.0,ph,NaN,2.0,1.0,-34.568157,-58.541866,Alicia Pinasco inmobiliaria,NaN
4368,63000.0,departamento,55.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0
4369,69000.0,casa,57.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0
4370,69000.0,casa,57.0,1.0,1.0,-34.572796,-58.520504,Inmobiliaria Viviana Villalobos,0.0


In [32]:
df[df2.columns].append(df2).reset_index(drop = True).to_csv('features_propiedades_4371.csv')